In [148]:
import pandas as pd
from datetime import datetime
from datetime import date
import json
import math
import xlrd
import numpy as np

pd.set_option('display.float_format','{:.4f}'.format)

hubs_dict = {
    'ARICA': 'IQUIQUE',
    'IQUIQUE': 'IQUIQUE',
    'ANTOFAGASTA': 'ANTOFAGASTA',
    'COPIAPO': 'COPIAPO',
    'COQUIMBO': 'COQUIMBO',
    'OVALLE': 'COQUIMBO',
    'ILLAPEL': 'CURAUMA',
    'LLAY LLAY': 'CURAUMA',
    'CURAUMA': 'CURAUMA',
    'SANTIAGO SUR': 'SANTIAGO SUR',
    'RANCAGUA': 'SANTIAGO SUR',
    'TALCA': 'TALCA',
    'CHILLAN': 'TALCAHUANO',
    'TALCAHUANO': 'TALCAHUANO',
    'LOS ANGELES': 'TALCAHUANO',
    'TEMUCO': 'TEMUCO',
    'VALDIVIA': 'VALDIVIA',
    'OSORNO': 'VALDIVIA',
    'PUERTO MONTT': 'PUERTO MONTT',
    'CASTRO': 'PUERTO MONTT',
    'COYHAIQUE': 'COYHAIQUE',
    'CALAMA': 'ANTOFAGASTA'
}

def stats(datos):
    
    lead_time = 7

    mean_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).mean().reset_index().rename(columns={'Venta en pallets': 'MEDIA'})
    stdev_no_group = datos.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST']).std().reset_index().rename(columns={'Venta en pallets': 'STD'})

    mean_no_group['MEDIA'] = mean_no_group.apply(lambda x: x.MEDIA * lead_time, axis=1)
    stdev_no_group['STD'] = stdev_no_group.apply(lambda x: x.STD * lead_time, axis=1)

    data_completa = mean_no_group.merge(stdev_no_group, on=['ID_SKU_VENTA', 'DESCR_CENDIST'])

    data_completa['COEF_VAR'] = data_completa.apply(lambda x: x['STD'] / x['MEDIA'] if x['MEDIA'] != 0 else 0, axis=1)

    data_completa['SS'] = data_completa.apply(lambda x: 1.65 * x['STD'], axis=1)

    data_completa['STOCK_TEORICO'] = data_completa.apply(lambda x: x['MEDIA'] + x['SS'], axis=1)

    data_completa['PORCENTAJE_SS_DEL_TOTAL'] = data_completa.apply(lambda x: x['SS'] / x['STOCK_TEORICO'] if x['MEDIA'] != 0 else 0, axis=1)

    porcentaje_ss_del_total = data_completa['SS'].sum() / data_completa['STOCK_TEORICO'].sum()
    
    return data_completa, porcentaje_ss_del_total

restricted = False
porcentaje = 100


In [149]:
datos_limpios = pd.read_csv('../data/datos_limpios_filtrados.csv')
datos_limpios = datos_limpios[(datos_limpios['DESCR_CENDIST'] != 'CERVECERA') & (datos_limpios['DESCR_CENDIST'] != 'MODELO')]
datos_limpios = datos_limpios.drop(columns='Unnamed: 0')

In [150]:
stats_no_agrupados, porcentaje_ss_no_agrupado = stats(datos_limpios)

In [151]:
if restricted:    
    FILE_AGRUPACIONES = f'../data/skuUnification/agrupacion_sku_{porcentaje}%.xlsx'
    sheets_dict = pd.read_excel(FILE_AGRUPACIONES, sheet_name=None)

    master = pd.DataFrame()
    for name in sheets_dict:

        df = sheets_dict[name]
        master = pd.concat([master, df])

    dict_sku_cd = {}
    master = master.rename(columns={'SKU': 'ID_SKU_VENTA', 'CD': 'DESCR_CENDIST'})

    for index, row in master.iterrows():

        if row['CANTIDAD HUB 3R'] != 0:
            dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 1

        else:
            dict_sku_cd[(row['ID_SKU_VENTA'], row['DESCR_CENDIST'])] = 0
            
    assign_hub = (lambda x: x['DESCR_CENDIST'] if (x['ID_SKU_VENTA'], x['DESCR_CENDIST']) in \
                            dict_sku_cd and dict_sku_cd[(x['ID_SKU_VENTA'], x['DESCR_CENDIST'])] == 0 \
                            else hubs_dict[x['DESCR_CENDIST']])

else:
    assign_hub = lambda x: hubs_dict[x['DESCR_CENDIST']]
    
datos_para_agrupar = datos_limpios.copy()
datos_para_agrupar['DESCR_CENDIST'] = datos_para_agrupar.apply(assign_hub, axis=1)
datos_agrupados = datos_para_agrupar.groupby(by=['ID_SKU_VENTA', 'DESCR_CENDIST', 'FECHA']).sum().reset_index()

In [152]:
stats_agrupados, porcentaje_ss_agrupado = stats(datos_agrupados)

In [ ]:
demandas = stats_agrupados.groupby(by=['ID_SKU_VENTA']).sum().reset_index()[['ID_SKU_VENTA', 'MEDIA']]
# demandas

In [153]:
no_agrupado_agregado = stats_no_agrupados.groupby(by=['DESCR_CENDIST']).sum()[['MEDIA', 'SS', 'STOCK_TEORICO']]
no_agrupado_agregado = no_agrupado_agregado.reset_index()
# no_agrupado_agregado

In [154]:
agrupado_agregado = stats_agrupados.groupby(by=['DESCR_CENDIST']).sum()[['MEDIA', 'SS', 'STOCK_TEORICO']]
agrupado_agregado = agrupado_agregado.reset_index()
# agrupado_agregado

In [187]:
no_agrupado_agregado

,DESCR_CENDIST,MEDIA,SS,STOCK_TEORICO
0,ANTOFAGASTA,594.1741,1995.3679,2589.5420
1,ARICA,293.7469,1068.2682,1362.0151
2,CALAMA,299.2557,1050.4151,1349.6708
3,CASTRO,174.2444,669.9235,844.1680
4,CHILLAN,516.2376,1796.6785,2312.9161
5,COPIAPO,426.2447,1504.2362,1930.4809
6,COQUIMBO,627.7351,2236.5164,2864.2515
7,COYHAIQUE,123.4891,495.6552,619.1443
8,CURAUMA,1774.7444,5245.7183,7020.4627
9,ILLAPEL,100.1641,479.5967,579.7609


In [186]:
agrupado_agregado

,DESCR_CENDIST,MEDIA,SS,STOCK_TEORICO
0,ANTOFAGASTA,893.4298,2778.9779,3672.4078
1,COPIAPO,426.2447,1504.2362,1930.4809
2,COQUIMBO,809.7393,2763.4216,3573.1609
3,COYHAIQUE,123.4891,495.6552,619.1443
4,CURAUMA,2610.0967,7612.2449,10222.3415
5,IQUIQUE,720.5237,2339.6877,3060.2114
6,PUERTO MONTT,650.6805,2104.9564,2755.6369
7,SANTIAGO SUR,3249.7340,9468.2010,12717.9350
8,TALCA,1004.3075,3227.0269,4231.3344
9,TALCAHUANO,2270.0388,6783.7577,9053.7964


In [156]:
delta_pallets = cds_no_agrupados['STOCK_TEORICO'].sum() - cds_agrupados['STOCK_TEORICO'].sum()
delta_pallets

2843.9188887512064

In [ ]:
if restricted:
    diff = no_agrupado_agregado.merge(agrupado_agregado, on=['DESCR_CENDIST'], how='outer')
    diff = diff.fillna(0)
    diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']
    deltaStocks = diff[['DESCR_CENDIST', 'DELTA']]
    
    OUTPUT_FILE_STOCKS = f'../data/Agrupación {porcentaje}%/inventario_final_agrupacion_{porcentaje}.csv'
    
    
    OUTPUT_FILE_DELTAS = f'../data/Agrupación {porcentaje}%/deltas_inventario_agrupacion_{porcentaje}.csv'
    deltaStocks.to_csv(OUTPUT_FILE_DELTAS)
    
    

In [ ]:
if not restricted:
    hubs = list(set(hubs_dict.values()))

    diff = no_agrupado_agregado.merge(agrupado_agregado, how='outer', on=['DESCR_CENDIST'])
    diff = diff.fillna(0)
    diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']

    diff['NEW_STOCK'] = 0

    diff['NEW_STOCK_CD'] = diff['MEDIA_x']
    diff['NEW_STOCK_HUB'] = diff['MEDIA_x'] + diff['SS_y']

    diff = diff[['DESCR_CENDIST', 'NEW_STOCK_HUB', 'NEW_STOCK_CD']]

    hubsStocks = diff[diff['DESCR_CENDIST'].isin(hubs)].reset_index()
    hubsStocks = hubsStocks[['DESCR_CENDIST', 'NEW_STOCK_HUB']]
    hubsStocks = hubsStocks.rename(columns={'NEW_STOCK_HUB': 'STOCK'})
    # hubsStocks

    cdStocks = diff[~diff['DESCR_CENDIST'].isin(hubs)].reset_index()
    cdStocks = cdStocks[['DESCR_CENDIST', 'NEW_STOCK_CD']]
    cdStocks = cdStocks.rename(columns={'NEW_STOCK_CD': 'STOCK'})
    # cdStocks

    newStocks = pd.concat([hubsStocks, cdStocks])
    newStocks = newStocks.sort_values(by=['DESCR_CENDIST'])
    # newStocks
    
    OUTPUT_FILE_STOCKS = '../data/Agrupacion total variabilidad/inventario_final_agrupacion_variabilidad_total.csv'
    newStocks.to_csv()

    oldStocks = no_agrupado_agregado[['DESCR_CENDIST', 'STOCK_TEORICO']]
    oldStocks = oldStocks.rename(columns={'STOCK_TEORICO': 'STOCK'})
    # oldStocks

    deltaStocks = oldStocks.merge(newStocks, on='DESCR_CENDIST')
    deltaStocks['DELTA'] = deltaStocks['STOCK_y'] - deltaStocks['STOCK_x'] 
    deltaStocks
    deltaStocks = deltaStocks[['DESCR_CENDIST', 'DELTA']]
    #deltaStocks

    OUTPUT_FILE_DELTAS = '../data/Agrupacion total variabilidad/deltas_inventario_agrupacion_variabilidad_total.csv'
    deltaStocks.to_csv(OUTPUT_FILE_DELTAS)

hubs = list(set(hubs_dict.values()))

diff = no_agrupado_agregado.merge(agrupado_agregado, how='outer', on=['DESCR_CENDIST'])
diff = diff.fillna(0)
diff['DELTA'] = diff['STOCK_TEORICO_y'] - diff['STOCK_TEORICO_x']

diff['NEW_STOCK'] = 0

diff['NEW_STOCK_CD'] = diff['MEDIA_x']
diff['NEW_STOCK_HUB'] = diff['MEDIA_x'] + diff['SS_y']

diff = diff[['DESCR_CENDIST', 'NEW_STOCK_HUB', 'NEW_STOCK_CD']]
# diff

hubsStocks = diff[diff['DESCR_CENDIST'].isin(hubs)].reset_index()
hubsStocks = hubsStocks[['DESCR_CENDIST', 'NEW_STOCK_HUB']]
hubsStocks = hubsStocks.rename(columns={'NEW_STOCK_HUB': 'STOCK'})
# hubsStocks

cdStocks = diff[~diff['DESCR_CENDIST'].isin(hubs)].reset_index()
cdStocks = cdStocks[['DESCR_CENDIST', 'NEW_STOCK_CD']]
cdStocks = cdStocks.rename(columns={'NEW_STOCK_CD': 'STOCK'})
# cdStocks

newStocks = pd.concat([hubsStocks, cdStocks])
newStocks = newStocks.sort_values(by=['DESCR_CENDIST'])
newStocks

oldStocks = no_agrupado_agregado[['DESCR_CENDIST', 'STOCK_TEORICO']]
oldStocks = oldStocks.rename(columns={'STOCK_TEORICO': 'STOCK'})
oldStocks

deltaStocks = oldStocks.merge(newStocks, on='DESCR_CENDIST')
deltaStocks['DELTA'] = deltaStocks['STOCK_y'] - deltaStocks['STOCK_x'] 
deltaStocks
deltaStocks = deltaStocks[['DESCR_CENDIST', 'DELTA']]

pd.set_option('display.float_format','{:.4f}'.format)
deltaStocks

OUTPUT_FILE_DELTAS = '../data/Agrupacion total variabilidad/deltas_inventario_agrupacion_variabilidad_total.csv'
deltaStocks.to_csv(OUTPUT_FILE_DELTAS)